In [1]:
import pandas as pd
import numpy as np
import sys

# df = pd.DataFrame(new_data, columns=column_names)
# df.set_index('tconst', inplace=True)
# df.to_csv('imdb_years_df.csv', sep=',')

In [28]:
# file headers
# tconst,nconst,primaryTitle,primaryName,primaryProfession
class TitleDictionary:

    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
        self.title_dict = self._create_title_dict()
        self.profession_dict = self._create_profession_dict()

    def _create_title_dict(self):
        title_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            if entry[1] in title_dict:
                title_dict[entry[1]].append(entry[2])
            else:
                title_dict[entry[1]] = [entry[2]]

        # print(title_dict['nm6551690']) # debug
        return title_dict 

    def _create_profession_dict(self):
        profession_dict = {}
        data = self.df.values.tolist()

        for entry in data:
            if entry[1] not in profession_dict:
                profession = entry[3]
                if entry[4] == 'actor':
                    profession+='_a'
                else:
                    profession+='_d'

                profession_dict[entry[1]] = profession

        # print(profession_dict['nm6446679']) # debug
        return profession_dict

file_path = 'imdb_network.csv'
test = TitleDictionary(file_path)

Bob Wehling_a


In [ ]:
#Graph Network Creation

'''
adj list
when adding edges
    if less than 2 movies in common, dont add the edge
    if actor->director, reverse the edge to director_actor
    if actor->actor, edge is bidirectional, so add the same edge for the actor

create graph is going to be a dict{dicts{}}, map of maps

add node
'''
class MovieNetwork:
    def __init__(self, name_movie_dict, nconst_ar_dr):
        self.graph = {} 
        self.name_movie_dict = name_movie_dict 
        self.nconst_ar_dr = nconst_ar_dr 

    def add_node(self, node):
        pass
    def add_edge(self, node1, node2, nconst_ar_dr, weight=1):
        pass
    def create_graph(self):

        return graph

In [11]:
# testing code 
print(18871 + 15753)
# file_path = 'imdb_network.csv'
# data = pd.read_csv(file_path)
# data = data.values.tolist()

# print(len(data))

# for i in data:
#     print(i)

34624
